# Miami-Dade Prop Appraiser Folio Scraper
This notebook takes a predefined list of folios and runs them through the Miami-Dade property appraiser. As it does so, it collects the JSON response and writes each one to its own separate file with the corresponding folio number for later analysis.

## Imports

In [1672]:
from bs4 import BeautifulSoup
import requests
from time import sleep
from random import randint
import json
import pandas as pd

## List & Function definitions

In [1676]:
succesful_scrape = []
failed_scrape = []
failed_scrape_e = []

In [1677]:
def scrape_url(folio):
    try:
        url = f"https://www.miamidade.gov/Apps/PA/PApublicServiceProxy/PaServicesProxy.ashx?Operation=GetPropertySearchByFolio&clientAppName=PropertySearch&folioNumber={folio}"
        raw_html = requests.get(url).content
        soup = BeautifulSoup(raw_html, "html.parser")
        json = open(f'{folio}.json','w')
        json.write(str(soup))
        json.close()
#        sleep(randint(1,3))
        succesful_scrape_transfer = []
        succesful_scrape_transfer.append(folio)
        succesful_scrape.append(succesful_scrape_transfer)
        return
    except Exception as e:
        failed_scrape_transfer = []
        failed_scrape_transfer_e = []
        failed_scrape_transfer.append(folio)
        failed_scrape_transfer_e.append(e)
        
        failed_scrape.append(failed_scrape_transfer)

In [ ]:
### Read in folios ###

## df = pd.read('folio_stuff.csv')

In [1678]:
## The 'counter' can be set to only return set amounts

# counter = 0
for folio in df.folio_num:
    scrape_url(folio)
    counter = counter + 1
#     if counter == 1000:
#         break

In [1679]:
print(len(failed_scrape_e))

0


In [1680]:
print(len(failed_scrape))

0


In [1681]:
print(len(succesful_scrape))

381


In [1682]:
## Turn succesful and failed scrapes into a DF ##

success_scrape_df = pd.DataFrame(succesful_scrape,columns=['folio_num'])
failed_scrape_df = pd.DataFrame(failed_scrape,columns=['folio_num'])

In [1683]:
success_scrape_df

,folio_num
0,0141100460210
1,0141161340001
2,0141020051810
3,0142070040350
4,0141090240750
...,...
376,0141040201670
377,0141100560201
378,0102060701060
379,0141040230560


In [1685]:
print(len(df))

381


In [1686]:
## These two lines merge the successful scrapes and regular DF, then removes all duplicates. Leaving only the unscraped DF
## This is incase the scraper gets interrupted for whatever reason.

df = pd.concat([df,success_scrape_df])
df = df.drop_duplicates(keep=False)

In [1690]:
### As a backup, write the remaining folios to scrape (and those that failed) to a csv file ###
df.to_csv('remaining_folios_to_scrape.csv',columns=['folio_num'])
failed_scrape_df.to_csv('failed_scrape.csv', header=['folio_num'])